## 4.8 APIによるランキング学習

In [ ]:
title_list = [
'泉質', '温泉', '温泉法', '温泉分析書', '万座温泉',
'伊東温泉', '吉野温泉', '塩江温泉', '塩津温泉',
'大子温泉', 
# '山田温泉',  # 山田温泉はユニークに決まらないのでとりあえずはずしてあります。
# 詳細は 2.1節を読んで下さい。
'川棚温泉', '指宿温泉',
'玉造温泉', '登別温泉', '花山温泉', '雲仙温泉',
'鳴子温泉', '鳴子温泉郷', '大歩危温泉'
]

### Wikipediaからテキストの一覧を作成

In [ ]:
# 必要ライブラリの導入
!pip install wikipedia | tail -n 1

In [ ]:
data_list = []

import wikipedia
wikipedia.set_lang("ja")
# wikipediaの記事の読み取り

for index, title in enumerate(title_list):
    print(index+1, title)
    text = wikipedia.page(title,auto_suggest=False).content
    item = {
        'app_id': index + 1,
        'title': title,
        'text': text
    }
    data_list.append(item)

## Discoveryにデータ投入

In [ ]:
# 必要ライブラリの導入
!pip install ibm_watson | tail -n 1

In [ ]:
# 資格情報の設定 (個別に設定します)

discovery_credentials = {
  "apikey": "xxxx",
  "iam_apikey_description": "xxxx",
  "iam_apikey_name": "xxxx",
  "iam_role_crn": "xxxx",
  "iam_serviceid_crn": "xxxx",
  "url": "xxxx"
}

In [ ]:
# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = '2019-04-30'

authenticator = IAMAuthenticator(discovery_credentials['apikey'])
discovery = DiscoveryV1(
    version=version,
    authenticator=authenticator
)
discovery.set_service_url(discovery_credentials['url'])

In [ ]:
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environment_id = discovery.list_environments().get_result()['environments'][1]['environment_id']
print('environment_id: ', environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id).get_result()['collections'][0]['collection_id']
print('collection_id: ' , collection_id)

# configuration_idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()['configurations'][0]['configuration_id']
print('configuration_id: ', configuration_id)

## 文書のロードと削除用関数 (4.6.2節)

In [ ]:
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称

def load_text( collection_id, sample_data, key_name):
    for item in sample_data:
        
        # itemごとにワークのjsonファイルを作成
        print(item)
        key = item.get(key_name)
        filename = str(key) + '.json'
        f = open(filename, 'w')
        json.dump(item, f)
        f.close()
        
        # 書き込み可能かのチェック
        collection = discovery.get_collection(environment_id, collection_id).get_result()
        proc_docs = collection['document_counts']['processing']
        while True:
            if proc_docs < 20:
                break
            print('busy. waiting..')
            time.sleep(10)
            collection = discovery.get_collection(environment_id, collection_id)
            proc_docs = collection['document_counts']['processing']

        # jsonファイル名を引数にDiscoveryへデータロード
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file = f)
        os.remove(filename)

In [ ]:
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション

def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']

    results = discovery.query(environment_id, collection_id, return_fields='id', count=doc_count).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print('deleting doc: id =' + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# 既存文書の全削除
delete_all_docs(collection_id)

In [ ]:
# wikipedia文書のロード
load_text(collection_id, data_list, 'app_id')

In [ ]:
# リスト4.8.1 

# トレーニングデータの全削除
discovery.delete_all_training_data(environment_id, collection_id)

## 自然言語問い合わせ

In [ ]:
# リスト4.8.2

# 自然言語問い合わせ
query_text = '温泉の特徴や泉質などの分類'
return_fields = 'app_id,title'

query_results = discovery.query(environment_id, collection_id, 
    natural_language_query=query_text,
    return_fields=return_fields).get_result()
res2 = query_results['results']

## ランキング学習

In [ ]:
# リスト4.8.3

# 問い合わせ結果表示 & examples配列の組立て
examples = []

for item in res2 :
    document_id = item['id']
    metadata = item['result_metadata']
    score = metadata['score']
    confidence = metadata['confidence']
    app_id = item['app_id']
    title = item['title']
    example = {
        'document_id': document_id,
         'cross_reference': app_id,
        'relevance': 0
    }
    print(document_id, title, app_id, score, confidence )
    examples.append(example)

In [ ]:
# リスト4.8.4 

# examples配列の完成
examples[0]['relevance'] = 10
examples[1]['relevance'] = 10

for example in examples:
    print(example)

In [ ]:
# リスト 4.8.5

# ランキング学習の実施
train_results = discovery.add_training_data(environment_id, collection_id, 
    natural_language_query=query_text, examples=examples).get_result()

In [ ]:
# リスト 4.8.6

# ランキング学習結果の確認
res2 = train_results['examples']
for item in res2:
    print(item)